# NO2 maps over India during lockdown period and business-as-usual

imports

In [1]:
import ee
import geemap.foliumap as geemap

# ee.Authenticate()
ee.Initialize()

Sentinel-5P Tropomi NO2 data

In [2]:
def get_NO2_map(start_date, end_date):
    NO2 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2').select('tropospheric_NO2_column_number_density').filterDate(start_date, end_date)
    return NO2

Get mean NO2 emissions in 2019 and 2020 for the period corresponding to lockdown

In [3]:
india_bbox = ee.Geometry.BBox(65.0865, 5.1299, 99.6275, 38.2344)

NO2_2019 = get_NO2_map('2019-03-25', '2019-05-03').mean().clip(india_bbox)
NO2_2020 = get_NO2_map('2020-03-25', '2020-05-03').mean().clip(india_bbox)

NO2_diff = NO2_2019.subtract(NO2_2020)

Interactive map for mean values over India

In [4]:
palette = ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
vis_params = {'min': 0, 'max': 0.00015, 'palette': palette}

m = geemap.Map(center=(22, 77), zoom=5)
m.addLayer(NO2_2019, vis_params, 'NO2 2019')
m.addLayer(NO2_2020, vis_params, 'NO2 2020')
m.addLayerControl()
m.add_colorbar(vis_params, label='NO2 (mol/m^2)')
m.to_html(filename='results/fig2a_2b.html', title='NO2 Maps', width='100%', height='880px')
m

Map of change in NO2

In [5]:
vis_params_diff = {'min': -0.00001, 'max': 0.0001, 'palette': palette}

m = geemap.Map(center=(22, 77), zoom=5)
m.addLayer(NO2_diff, vis_params_diff, 'NO2 diff')
m.addLayerControl()
m.add_colorbar(vis_params_diff, label='NO2 (mol/m^2)')
m.to_html(filename='results/fig2c.html', title='NO2 diff Map', width='100%', height='880px')
m

Delhi administrative boundary from Global Administrative Unit Layers

In [6]:
gaul = ee.FeatureCollection("FAO/GAUL/2015/level1")
delhi = gaul.filter(ee.Filter.eq('ADM1_NAME', 'Delhi'))
delhi = delhi.geometry()

Change in NO2 over Delhi

In [7]:
styleParams = {'fillColor': 'b5ffb4', 'color': 'black', 'width': 1.0}

m = geemap.Map(center=(28.6, 77.2), zoom=10)
m.addLayer(delhi, styleParams, 'Delhi')
m.addLayer(NO2_diff.clip(delhi), vis_params_diff, 'NO2 diff Delhi')
m.addLayerControl()
m.add_colorbar(vis_params_diff, label='NO2 (mol/m^2)')
m.to_html(filename='results/fig9.html', title='NO2 diff Map for Delhi', width='100%', height='880px')
m